In [25]:
import II
import NN
import h5py
import json
import time
import pyccl
import scipy
import numpy
from itertools import product
from matplotlib import pyplot
from matplotlib.gridspec import GridSpec

In [26]:
#Data path

TYPE = 'II'
PATH = '/pscratch/sd/y/yhzhang/TensorCloud/'

DATA_PATH = PATH + 'DATA/'
PLOT_PATH = PATH + 'PLOT/' + TYPE + '/'

In [27]:
#Load in Data

BIN_SIZE = 4
GRID_SIZE = 128
DATA_SIZE = 100000

with h5py.File(DATA_PATH + 'DATA.hdf5','r') as DATA:
    
    REDSHIFT_GRID = numpy.array(DATA['realizations/z'][:GRID_SIZE], dtype = 'float32')
    DATA_GRID = numpy.array(DATA['realizations/pdfs'][:DATA_SIZE, :BIN_SIZE, :GRID_SIZE], dtype = 'float32')

In [28]:
# COSMOLOGY

with open(DATA_PATH + 'COSMO.json', 'r') as cosmo:
    
    COSMO = json.load(cosmo)

COSMO_CCL = pyccl.Cosmology(
    h = COSMO['H'],
    w0 = COSMO['W0'],
    wa = COSMO['WA'], 
    n_s = COSMO['NS'], 
    A_s = COSMO['AS'],
    m_nu = COSMO['MNU'],  
    Neff = COSMO['NEFF'],
    T_CMB = COSMO['TCMB'], 
    Omega_k = COSMO['OMEGAK'], 
    Omega_c = COSMO['OMEGAC'], 
    Omega_b = COSMO['OMEGAB'], 
    matter_power_spectrum = 'halofit',  
    transfer_function = 'boltzmann_camb', 
    extra_parameters = {'camb': {'kmax': 100, 'lmax': 100, 'halofit_version': 'mead2020_feedback', 'HMCode_logT_AGN': 7.8}}
)

In [29]:
#Restrcture the data

PSI_GRID = numpy.mean(DATA_GRID, axis = 0)
SIGMA_GRID = numpy.std(DATA_GRID, axis = 0)
A_GRID = numpy.array(1 / (1 + REDSHIFT_GRID), dtype = 'float32')

CHI_GRID = COSMO_CCL.comoving_radial_distance(a = A_GRID)
PHI_GRID = PSI_GRID * COSMO_CCL.h_over_h0(a = 1 / (1 + REDSHIFT_GRID)) * COSMO['H'] * 100000 / scipy.constants.c

ELL_SIZE = 200
ELL_DATA = numpy.logspace(2, 4, ELL_SIZE + 1)
CHI_MESH, ELL_MESH = numpy.meshgrid(CHI_GRID, ELL_DATA)
SCALE_GRID = numpy.nan_to_num(numpy.divide(ELL_MESH + 1/2, CHI_MESH, out = numpy.zeros((ELL_SIZE + 1, GRID_SIZE)) + numpy.inf, where = CHI_MESH > 0))

POWER_GRID = numpy.zeros((ELL_SIZE + 1, GRID_SIZE))
for GRID_INDEX in range(GRID_SIZE):

    POWER_GRID[:,GRID_INDEX] = COSMO_CCL.linear_matter_power(k = SCALE_GRID[:,GRID_INDEX], a = A_GRID[GRID_INDEX])

GROWTH_RATE = pyccl.background.growth_rate(cosmo=COSMO_CCL,a = A_GRID)

In [31]:
#Define angular power spectrum

def c_m1_m2(amplitude, phi_m1_grid, phi_m2_grid, chi_grid, power_grid, growth_grid, type):

    pii = scipy.interpolate.interp1d(chi_grid, power_grid, kind = type)
    
    phi_m1 = scipy.interpolate.interp1d(chi_grid, phi_m1_grid/growth_grid, kind = type)
    
    phi_m2 = scipy.interpolate.interp1d(chi_grid, phi_m2_grid/growth_grid, kind = type)
    
    function = lambda chi: phi_m1(chi) * phi_m2(chi) * pii(chi) / (chi**2)
    
    c_m1_m2_ell = amplitude * scipy.integrate.fixed_quad(func = function, a = chi_grid.min(), b = chi_grid.max(), n = 500)[0]

    return c_m1_m2_ell

In [32]:
#Compare different numeric equations
AMPLITUDE = (0.0134**2)*(COSMO['OMEGAM'] + COSMO['OMEGAN'])**2 * (1 + 3 / (2 * ELL_DATA + 1)) * (1 + 1 / (2 * ELL_DATA + 1)) * (1 - 1 / (2 * ELL_DATA + 1)) * (1 - 3 / (2 * ELL_DATA + 1))
C_DATA = numpy.zeros((BIN_SIZE, BIN_SIZE, ELL_SIZE + 1))
C_DATA1 = numpy.zeros((BIN_SIZE, BIN_SIZE, ELL_SIZE + 1))
C_DATA2 = numpy.zeros((BIN_SIZE, BIN_SIZE, ELL_SIZE + 1))
C_DATA3 = numpy.zeros((BIN_SIZE, BIN_SIZE, ELL_SIZE + 1))
C_CCL = numpy.zeros((BIN_SIZE, BIN_SIZE, ELL_SIZE + 1))

T0 = time.time()
C_DATA = II.function(amplitude = AMPLITUDE, phi_grid = PHI_GRID, chi_grid = CHI_GRID, power_grid = POWER_GRID,growth_grid=GROWTH_RATE)

T1 = time.time()
print(T1 - T0)
POWER_test_GRID=numpy.zeros((ELL_SIZE + 1, GRID_SIZE))
IAb=numpy.ones(len(REDSHIFT_GRID))
for i in range(len(REDSHIFT_GRID)):
    IAb[i] = 0.0134 * (COSMO['OMEGAM'] + COSMO['OMEGAN']) /GROWTH_RATE[i]

for i in range(ELL_SIZE + 1):
    for j in range(GRID_SIZE):
        POWER_test_GRID[i][j] =POWER_GRID[i][j]*(IAb[j]**2)
C_DATA_test = NN.function(amplitude = 1, phi_grid = PHI_GRID, chi_grid = CHI_GRID, power_grid = POWER_test_GRID, redshift_grid = REDSHIFT_GRID)
print(C_DATA_test[0][0]/C_DATA[0][0])


2.4069085121154785
[1.00004307 1.00003502 1.00002749 1.00001987 1.00001162 1.00000245
 0.99999226 0.9999812  0.9999696  0.99995788 0.99994654 0.99993607
 0.99992686 0.99991914 0.99991298 0.99990816 0.99990424 0.99990063
 0.99989669 0.99989188 0.999886   0.99987915 0.99987192 0.99986496
 0.99985889 0.99985398 0.99985021 0.99984721 0.99984454 0.99984177
 0.99983867 0.99983514 0.99983122 0.99982707 0.99982298 0.99981929
 0.99981621 0.99981377 0.9998117  0.9998097  0.99980743 0.99980482
 0.999802   0.99979923 0.99979681 0.99979479 0.99979305 0.99979133
 0.99978947 0.99978748 0.99978552 0.9997837  0.99978206 0.99978053
 0.99977901 0.99977748 0.99977598 0.99977455 0.99977318 0.99977186
 0.99977057 0.99976932 0.99976811 0.99976693 0.99976577 0.99976464
 0.99976354 0.99976248 0.99976145 0.99976045 0.99975949 0.99975856
 0.99975767 0.99975679 0.99975594 0.99975511 0.99975429 0.9997535
 0.99975273 0.99975197 0.99975123 0.99975051 0.99974981 0.99974912
 0.99974845 0.9997478  0.99974716 0.99974654